In [307]:
import sqlite3
import pandas as pd

from sqlite3 import (
    DatabaseError,
    IntegrityError,
    OperationalError
)

from typing import (
    Dict,
    List,
    Optional,
    Tuple,
    Union
)

from collections import OrderedDict

In [30]:
database = 'file.database.db'

In [232]:
tables: OrderedDict = OrderedDict({
    'file_id':'TEXT',    # PRIMARY KEY
    'rel_path':'TEXT',
    'file_date':'TEXT',
    'acq_date':'TEXT',
    'sub_id':'TEXT',
    'ses_id':'TEXT',
    'bids_name':'TEXT'
})

In [210]:
def create_db_test_1(database: str,
                     tables: OrderedDict
                     ) -> str:
    """Test database creation function
    """
    # Create/access database
    conn = sqlite3.connect(database)
    c = conn.cursor()

    # Construct database tables
    for i in range(1,len(tables)):
        table_name: str = list(tables.keys())[i]
        new_field: str = list(tables.keys())[i]
        field_type: str = tables.get(list(tables.keys())[i],'NULL')
        
        # Create database tables
        try:
            tn: str = table_name
            # Primary Key
            nf1: str = list(tables.keys())[0]
            ft1: str = tables.get(list(tables.keys())[0],'NULL')
            # Child Key
            nf2: str = new_field
            ft2: str = field_type

            query: str = f"CREATE TABLE {tn} ({nf1} {ft1} PRIMARY KEY, {nf2} {ft2})"
            c.execute(query)
        except OperationalError:
            # print(table_name)
            continue
    
    # Commit changes and close the connection
    conn.commit()
    conn.close()
    return database

In [233]:
create_db_test_1(database,tables)

'file.database.db'

In [236]:
info: Dict[str,str] = {
    'file_id': '0000002',    # PRIMARY KEY
    'rel_path':'/Users/adebayobraimah/Desktop/projects/sql_db/file.py',
    'file_date':'2021-05-22',
    'acq_date':'2021-05-22',
    'sub_id':'001',
    'ses_id':'',
    'bids_name':"sub-001_run-01_T1w"
}

In [149]:
def insert_db_test(database: str,
                    tables: OrderedDict,
                    info: Dict[str,str]
                    ) -> str:
    """Inserts rows into existing database tables.
    """
    # Access database
    conn = sqlite3.connect(database)
    c = conn.cursor()

    # Insert new rows into database tables
    for i in range(1,len(tables)):
        table_name: str = list(tables.keys())[i]
        new_field: str = list(tables.keys())[i]

        tn: str = table_name
        p_key: str = list(tables.keys())[0]
        col: str = new_field

        p_val: str = info[list(tables.keys())[0]]
        col_val: str = info.get(list(tables.keys())[i],'NULL')
        
        query: str = f"INSERT INTO {tn} ({p_key},{col}) VALUES( ?,? )"

        try:
            c.execute(query, (p_val,col_val))
        except IntegrityError:
            continue
    
    conn.commit()
    conn.close()
    return database

In [237]:
insert_db_test(database,tables,info)

'file.database.db'

In [160]:
# TODO:
#   * Add method/function to export database to spreadsheet/dataframe

In [175]:
def get_len_rows_test(database: str, 
                    tables: OrderedDict
                    ) -> int:
    """Gets number of rows in a databases' table.
    """
    # Access database
    conn = sqlite3.connect(database)
    c = conn.cursor()

    # Perform database query
    query: str = f"SELECT COUNT(*) from {list(tables.keys())[1]}"
    c.execute(query)

    result: int = c.fetchone()[0]
    return result

In [189]:
get_len_rows_test(database,tables)

2

In [177]:
def get_file_id(database: str, 
                tables: OrderedDict
                ) -> str:
    """Returns new file_id for file that does not yet exist in the database.
    """
    file_id: int = get_len_rows_test(database, tables) + 1
    fild_id: str = str(file_id)
    # zeropad number here
    return file_id

In [190]:
get_file_id(database,tables)

3

In [209]:
def update_table_row(database: str,
                    prim_key: str,
                    table_name: str, 
                    col_name: str, 
                    value: Optional[Union[int,str]]
                    ) -> str:
    """Updates a row in a table in some given database.
    """
    # Access database
    conn = sqlite3.connect(database)
    c = conn.cursor()

    # Perform database table update
    query: str = f"UPDATE {table_name} SET {col_name} = ? WHERE {list(tables.keys())[0]} = ?"

    c.execute(query, (value,prim_key))

    conn.commit()
    conn.close()
    return database

In [238]:
update_table_row(database,
                prim_key='0000002',
                table_name='sub_id',
                col_name='sub_id',
                value='002')

'file.database.db'

In [288]:
def export_dataframe(database: str,
                    tables: OrderedDict
                    ) -> pd.DataFrame:
    """Exports joined tables from the input database.
    """
    # Access database
    conn = sqlite3.connect(database)

    df_list: List = []

    for i in range(1,len(tables)):
        table = list(tables.keys())[i]
        df_tmp: pd.DataFrame = pd.read_sql_query(f"SELECT * FROM {table}", conn)

        if i == 1:
            pass
        else:
            df_tmp = df_tmp.drop(labels=list(tables.keys())[0],axis=1)

        df_list.append(df_tmp)

    df: pd.DataFrame = pd.concat(df_list,axis=1,join='outer')

    return df

In [289]:
export_dataframe(database,tables)

,file_id,rel_path,file_date,acq_date,sub_id,ses_id,bids_name
0,0000001,/Users/adebayobraimah/Desktop/projects/sql_db/...,2021-05-22,2021-05-22,001,,sub-001_run-01_T1w
1,0000002,/Users/adebayobraimah/Desktop/projects/sql_db/...,2021-05-22,2021-05-22,002,,sub-001_run-01_T1w


In [275]:
list(tables.keys())[0]

'file_id'

In [333]:
def export_scans_dataframe(database: str,
                            info: Dict[str,str],
                            raise_exec: bool = False,
                            *args: str
                            ) -> pd.DataFrame:
    """Exports a dataframe provided table/column IDs from the input ordered dictionary that corresponds to the table/colmns in the provided database.
    """
    # Access database
    conn = sqlite3.connect(database)
    c = conn.cursor()

    df_list: List = []

    for i in args:
        table = str(i)

        query: str = f"SELECT count(name) FROM sqlite_master WHERE type='table' AND name='{table}'"

        c.execute(query)

        if c.fetchone()[0] == 1:
            pass
        else:
            if raise_exec:
                raise DatabaseError(f"Table {table} does not exist in database")
            continue
        
        df_tmp: pd.DataFrame = pd.read_sql_query(f"SELECT * FROM {table}", conn)
        df_tmp = df_tmp.drop(labels=list(tables.keys())[0],axis=1)
        df_list.append(df_tmp)

    df: pd.DataFrame = pd.concat(df_list,axis=1,join='outer')

    return df

In [334]:
export_scans_dataframe(database,tables,False,'sub_id','ses_id','bids_name','test')

,sub_id,ses_id,bids_name
0,001,,sub-001_run-01_T1w
1,002,,sub-001_run-01_T1w


## Get `relative` path for `file_name`

In [5]:
import os
import pathlib

In [3]:
study_dir = "/Users/adebayobraimah/Desktop/projects/convert_source/test.data/study.images"
file_name = "/Users/adebayobraimah/Desktop/projects/convert_source/test.data/study.images/IRC287H009/IRC287H-9_AXIAL_3_3.nii.gz"

In [12]:
path_sep = os.path.sep
dir_tmp = str(pathlib.Path(study_dir).parents[0])
dir_tmp

'/Users/adebayobraimah/Desktop/projects/convert_source/test.data'

In [15]:
file_name.replace(dir_tmp + path_sep,"." + path_sep)

'./study.images/IRC287H009/IRC287H-9_AXIAL_3_3.nii.gz'